In [86]:
from typing import List
from collections import defaultdict, Counter

class Solution:
    def countPairs(self, n: int, edges: List[List[int]], queries: List[int]) -> List[int]:
        # 每个点对在 edges中出现的数目
        count = defaultdict(int)
        for (a, b) in edges:
            count[(a, b)] += 1
        
        # 每个点有多少边相连
        graph = defaultdict(int)
        for a, b in edges:
            graph[a] += 1
            graph[b] += 1
        
        nums = []
        for i in range(1, n+1):
            ways = graph[i]
            for j in range(i+1, n+1):
                ways = graph[i] + graph[j] - count[(i, j)] - count[(j, i)]
                nums.append(ways)
        
        res = [0] * len(queries)
        for k, q in enumerate(queries):
            pass
        return res

In [95]:
'''
题目名 5683. 统计点对的数目
思路：统计每一个节点的度，将所有度数升序排序，对于每一个查询，在有序序列中找和超过门限值的点对
有多少对，可以枚举第一个点的度数，然后二分找第二个点下标的下边界，将点对的数量累加，但是这样
没有考虑点对之间直连边被算了两次的情况，会多累加一些不合法的点对，后面在枚举边，对于边(a, b) (其中a<b)
可能会影响(a, b)这个点对的cnt值，所以可以计算这个点对的实际的cnt = d[a] + d[b] - (a,b)边的条数
如果实际的cnt不符合当前的查询条件，就从当前的答案计数中扣去1，总时间复杂度 O(Q*(N*log(N) + M))，其中N M分别是点数和边数
Q是询问数
'''

from collections import Counter

class Solution:
    def countPairs(self, n: int, edges: List[List[int]], queries: List[int]) -> List[int]:
        d = [0] * (n+1)     # 每一个点的度
        edge_cnt = Counter()
        for a, b in edges:
            if a > b:
                a, b = b, a

            d[a] += 1
            d[b] += 1
            edge_cnt[(a, b)] += 1
        
        arr = [d[node] for node in range(1, n+1)]
        arr.sort()
        
        q_cnt = len(queries)
        ans = [0] * q_cnt
        for q_idx in range(q_cnt):
            q_val = queries[q_idx]
            
            for i in range(n-1):
                degree = arr[i]
                # 二分找第一个大于q_val - degree的位置
                pos = -1
                l, r = i+1, n-1
                while l <= r:
                    mid = (l+r) // 2
                    if arr[mid] > q_val - degree:
                        pos = mid
                        r = mid - 1
                    else:
                        l = mid + 1

                if pos != -1:
                    # 这里对点对计数时候是没有考虑点对之间的边被算了两次的，会多加点对，后面再扣掉
                    ans[q_idx] += (n-1) - pos + 1

            # 把多算的点对扣掉
            for (a, b), cnt in edge_cnt.items():
                if d[a] + d[b] > q_val and d[a] + d[b] - cnt <= q_val:
                    ans[q_idx] -= 1
        return ans

In [218]:
from collections import Counter
from typing import List

class Solution:
    def countPairs(self, n: int, edges: List[List[int]], queries: List[int]) -> List[int]:
        count = [0] * (n+1)
        graph = defaultdict(int)
        for a, b in edges:
            count[a] += 1
            count[b] += 1
            if a < b:
                graph[(a, b)] += 1
            else:
                graph[(b, a)] += 1
        
        # 每条边的入度按照从小到大进行排序
        indeg = sorted(count[1:])
        ans = [-1] * len(queries)
        # 使用双指针的方法
        for idx, q in enumerate(queries):
            cnt = 0
            right = n-1
            # 遍历左边界
            for left in range(n):
                # 只有当条件满足的时候，才能移动右边界
                if left >= right:
                    cnt += n - left - 1
                else:
                    # 因为left == right, idx相同的时候不能算入，所以要减去1
                    while right > left and indeg[right] + indeg[left] > q:
                        right -= 1
                    # 这里为什么要 -1，因为当退出的时候，indeg[right] + indeg[left]是不符合条件的，所以要减 1
                    cnt += n - right - 1
                    
            # 去除不满足要求的边
            for (a, b), k in graph.items():
                if count[a] + count[b] > q and count[a] + count[b] - k <= q:
                    cnt -= 1
            ans[idx] = cnt
        return ans

In [219]:
solution = Solution()
solution.countPairs(5, [[4,5],[1,3],[1,4]], [0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,2])

[10, 8, 10, 10, 8, 8, 10, 10, 10, 10, 8, 10, 10, 8, 10, 8, 8, 3]

In [ ]:
[10,8,10,10,8,8,10,10,10,10,8,10,10,8,10,8,8,3]